In [17]:
import pandas as pd 

df_fidelity = pd.read_csv("n_mfp_fidelity.csv")
df_goldman = pd.read_csv("n_mfp_goldman.csv")
df_jpmorgan = pd.read_csv("n_mfp_jpmorgan.csv")

df_fidelity['Source'] = 'Fidelity'
df_jpmorgan['Source'] = 'JPMorgan'
df_goldman['Source'] = 'Goldman Sachs'


In [18]:
len(df_fidelity)

108

In [19]:
cols_to_keep = ['Filing Date', 'WAM (days)', 'Net Assets', 'Source']

df_fidelity = df_fidelity[cols_to_keep]
df_jpmorgan = df_jpmorgan[cols_to_keep]
df_goldman = df_goldman[cols_to_keep]


df_all = pd.concat([df_fidelity, df_jpmorgan, df_goldman], ignore_index=True)


In [20]:
df_all.head()

,Filing Date,WAM (days),Net Assets,Source
0,2016-07-08,37.0,5.543205e+09,Fidelity
1,2016-08-04,29.0,5.726993e+09,Fidelity
2,2016-09-08,43.0,6.602602e+10,Fidelity
3,2016-10-07,49.0,7.093520e+10,Fidelity
4,2016-11-07,48.0,7.363233e+10,Fidelity


In [21]:
df_all['Filing Date'] = pd.to_datetime(df_all['Filing Date'], errors='coerce')


In [22]:
# Create a Month column
df_all['Month'] = df_all['Filing Date'].dt.to_period('M')

# Group by Month and calculate weighted average
wam_index = df_all.groupby('Month').apply(
    lambda x: (x['WAM (days)'] * x['Net Assets']).sum() / x['Net Assets'].sum()
).reset_index(name='Weighted Avg Maturity')

# Optional: convert Period to Timestamp (e.g. '2023-05' to datetime)
wam_index['Month'] = wam_index['Month'].dt.to_timestamp()


In [23]:
print(wam_index)


         Month  Weighted Avg Maturity
0   2016-07-01              39.927583
1   2016-08-01              34.917873
2   2016-09-01              41.687256
3   2016-10-01              41.190374
4   2016-11-01              47.418204
..         ...                    ...
104 2025-03-01              33.568145
105 2025-04-01              27.175088
106 2025-05-01              33.131922
107 2025-06-01              33.032234
108 2025-07-01              12.000000

[109 rows x 2 columns]


In [24]:
print(wam_index['Month'].min(), "→", wam_index['Month'].max())
print("Total rows:", len(wam_index))


2016-07-01 00:00:00 → 2025-07-01 00:00:00
Total rows: 109


In [25]:
wam_index.tail()

,Month,Weighted Avg Maturity
104,2025-03-01,33.568145
105,2025-04-01,27.175088
106,2025-05-01,33.131922
107,2025-06-01,33.032234
108,2025-07-01,12.000000


In [26]:
# Keep only the expected date range
mask = (wam_index['Month'] >= '2016-06-01') & (wam_index['Month'] <= '2025-06-01')
wam_index = wam_index.loc[mask].reset_index(drop=True)


In [27]:
len(wam_index)

108

In [28]:
wam_index.tail()

,Month,Weighted Avg Maturity
103,2025-02-01,37.900955
104,2025-03-01,33.568145
105,2025-04-01,27.175088
106,2025-05-01,33.131922
107,2025-06-01,33.032234


In [29]:
wam_index.to_csv('wam_index.csv', index=False)
